<a href="https://colab.research.google.com/github/hantswilliams/HHA_504_2023/blob/main/WK3/code/class_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading in packages

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

# Creating a temporary + local DB using SQLITE

In [29]:
conn = sqlite3.connect('costs.db')
c = conn.cursor()

## Now lets create a new table called stonybrook with the costs.db file

In [3]:
c.execute("""
            CREATE TABLE stonybrook
                (
                    hospital_name text,
                    insurance_type text,
                    code text,
                    code_description text,
                    cost_negotiated real,
                    cost_minimum real,
                    cost_maximum real
                );
          """)

conn.commit()

## Lets now confirm that the new table stonybrook has been created within the database costs.db

In [7]:
c.execute('''
  SELECT name
  FROM sqlite_master
  WHERE type='table';
  ''')

c.fetchall()

#for value in c.fetchall():
#    print(value)

[('stonybrook',)]

In [8]:
c.execute('''
  SELECT * FROM stonybrook;
''')

print(c.fetchall())

[]


# Inserting in data

In [10]:
sql_query = """

INSERT INTO stonybrook (
  'hospital_name',
  'insurance_type',
  'code',
  'code_description',
  'cost_negotiated',
  'cost_minimum',
  'cost_maximum'
  )
  values (
    'southampton hospital',
    'aetna',
    '99214',
    'office visit',
    100.00,
    1.00,
    1000.00
  );

"""

print(sql_query)



INSERT INTO stonybrook (
  'hospital_name', 
  'insurance_type',
  'code',
  'code_description',
  'cost_negotiated',
  'cost_minimum',
  'cost_maximum'
  ) 
  values (
    'southampton hospital',
    'aetna',
    '99214',
    'office visit',
    '100.00',
    '1.00',
    '1000.00'
  ); 




In [12]:
c.execute(sql_query)
conn.commit()

# Lets now check that the one row has been inserted using sqlite

In [13]:
sql_query_2 = """

select *
from stonybrook;

"""

c.execute(sql_query_2)
print(c.fetchall())

[('southampton hospital', 'aetna', '99214', 'office visit', 100.0, 1.0, 1000.0), ('southampton hospital', 'aetna', '99214', 'office visit', 100.0, 1.0, 1000.0)]


Lets now check using PANDAS and return the response as a datframe

Before we now try and connect with sqlalchemy, we need to stop our connection from sqlite

In [23]:
# conn.close()

In [24]:
# ## create engine to connect to our sqlite DB
engine = create_engine('sqlite:///costs.db')

In [30]:
pd.read_sql_query("select * from stonybrook;", conn)


,hospital_name,insurance_type,code,code_description,cost_negotiated,cost_minimum,cost_maximum
0,southampton hospital,aetna,99214,office visit,100.0,1.0,1000.0
1,southampton hospital,aetna,99214,office visit,100.0,1.0,1000.0


# Load in some real data from SBM - costs per CPT codes

In [31]:
# Load in csv data from WK3/data/stonybrook/stonybrook.csv

df = pd.read_csv('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/stonybrook/stonybrook.csv')

# Shape
df.shape

# Columns
df.columns

## Create new column that goes from wide to long, where we keep columns 0:8 and then melt the rest into a new
## column called 'insurance_type'

columnNames = list(df)
idVars = columnNames[:8]
valueVars = columnNames[8:]

stonybrook_modified = df.melt(id_vars=idVars, value_vars=valueVars)

stonybrook_modified.columns

stonybrook_modified.rename(columns={'HospCode':'hospital_name',
            'variable':'insurance_type',
            'Code':'code',
            'Description':'code_description',
             'value':'cost_negotiated',
             'Minimum Negotiated Charge':'cost_minimum',
             'Maximum Negotiated Charge': 'cost_maximum'}, inplace=True)

stonybrook_modified['insurance_type'].value_counts()

stonybrook_modified

,code,code_description,Type,Package/Line_Level,Gross charge,Discounted cash price,De-identified min contracted rate,De-identified max contracted rate,insurance_type,cost_negotiated
0,10004,"FINE NEEDLE ASPIRATION BIOPSY, EACH ADDITIONAL...",Outpatient,Line,718.67,718.67,2155.50,2155.50,Derived contracted rate,2155.50
1,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,2061.25,2061.25,87.02,4782.00,Derived contracted rate,2308.44
2,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Package,2061.25,2061.25,87.02,4782.00,Derived contracted rate,2308.44
3,10006,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,728.54,728.54,58.55,1761.25,Derived contracted rate,909.90
4,10009,FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...,Outpatient,Line,2823.01,2823.01,446.28,6017.59,Derived contracted rate,4219.31
...,...,...,...,...,...,...,...,...,...,...
377785,30981203,R&B SICU 09W1 ICR PT-Z,NaN,NaN,12090.00,12090.00,NaN,NaN,Veteran Family-Commercial other,NaN
377786,30993018,R&B TICR 09W2 - PT-I,NaN,NaN,13590.00,13590.00,NaN,NaN,Veteran Family-Commercial other,NaN
377787,33500000,R&B REV PICU 11S2 PT-X,NaN,NaN,8165.00,8165.00,NaN,NaN,Veteran Family-Commercial other,NaN
377788,30333058,ROPH HOSP.OBSERVATION PT-T,NaN,NaN,8165.00,8165.00,NaN,NaN,Veteran Family-Commercial other,NaN


In [32]:
stonybrook_modified.to_sql('stonybrook', conn, if_exists='replace')

377790

## Lets now test that this work!



In [41]:
query = """
  select *
  from stonybrook
  where code = '003X'
  limit 100;
"""

response = pd.read_sql(query, conn)
response

,index,code,code_description,Type,Package/Line_Level,Gross charge,Discounted cash price,De-identified min contracted rate,De-identified max contracted rate,insurance_type,cost_negotiated
0,6042,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Derived contracted rate,53264.34
1,13752,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,1199-Commercial other,NaN
2,21462,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Aetna-Medicare Advantage HMO,NaN
3,29172,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Aetna-Commercial HMO/POS,NaN
4,36882,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Aetna-Commercial PPO/Open Access,NaN
5,44592,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Aetna-Commercial other,NaN
6,52302,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Empire Health-Commercial other,92452.99
7,60012,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Empire Health-Commercial PPO/Open Access,36715.25
8,67722,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,BlueCross BlueShield-Commercial other,NaN
9,75432,003X,Bone Marrow Transplant,Inpatient,Line,557046.21,557046.21,35620.0,92452.99,Beacon Health-Commercial other,NaN
